# Module 2: The ReAct Pattern

**Duration:** ~15 minutes  
**Goal:** Understand multi-step reasoning with Think → Act → Observe loops

---

## Beyond Single Calls

In the last module, we saw tool calling: ask a question, call a tool, get an answer.

But real questions aren't that simple:

```
"Compare Apple and NVIDIA's stock performance.
 Which one has grown more relative to its 52-week high?"
```

To answer this, you need to:
1. Get Apple's current price
2. Get Apple's 52-week high
3. Get NVIDIA's current price
4. Get NVIDIA's 52-week high
5. Calculate the percentage difference for each
6. Compare and explain

That's not one tool call. That's a **chain of reasoning with multiple steps**.

This is where the **ReAct pattern** comes in.

## What is ReAct?

**ReAct** stands for **Reason + Act**. It's a pattern where the agent alternates between thinking and doing.

```
┌─────────────────────────────────────────────────────┐
│                  THE REACT LOOP                      │
│                                                      │
│    ┌──────────┐     ┌──────────┐     ┌──────────┐  │
│    │  THINK   │ ──→ │   ACT    │ ──→ │ OBSERVE  │  │
│    │          │     │          │     │          │  │
│    │ "What do │     │ Call a   │     │ See the  │  │
│    │  I need  │     │ tool     │     │ result   │  │
│    │  next?"  │     │          │     │          │  │
│    └──────────┘     └──────────┘     └──────────┘  │
│          ↑                                 │        │
│          └─────────────────────────────────┘        │
│                    (repeat until done)              │
└─────────────────────────────────────────────────────┘
```

- **Think:** The agent reasons about what to do next
- **Act:** It calls a tool to get information
- **Observe:** It sees the result
- Then it loops back to **Think** — what do I do with this? Do I need more?

---

## Setup

In [1]:
# Install smolagents if needed
!pip install smolagents -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


In [2]:
from smolagents import CodeAgent, tool
from smolagents import OpenAIServerModel
from smolagents.monitoring import LogLevel
import getpass

In [3]:
# Enter your API key
API_KEY = getpass.getpass("Enter your OpenAI API key: ")
model = OpenAIServerModel("gpt-4o-mini", api_key=API_KEY)
print("Model ready!")

Enter your OpenAI API key: ··········
Model ready!


---

## Creating Our Tools

Let's set up three tools for stock analysis.

In [4]:
@tool
def get_stock_price(ticker: str) -> float:
    """Get the current price for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA', 'MSFT')

    Returns:
        The current stock price as a float
    """
    prices = {"AAPL": 178.50, "NVDA": 875.30, "MSFT": 378.90}
    return prices.get(ticker.upper(), 0.0)

@tool
def get_52_week_high(ticker: str) -> float:
    """Get the 52-week high price for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA', 'MSFT')

    Returns:
        The 52-week high price as a float
    """
    highs = {"AAPL": 199.62, "NVDA": 974.00, "MSFT": 420.82}
    return highs.get(ticker.upper(), 0.0)

@tool
def get_52_week_low(ticker: str) -> float:
    """Get the 52-week low price for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA', 'MSFT')

    Returns:
        The 52-week low price as a float
    """
    lows = {"AAPL": 164.08, "NVDA": 393.05, "MSFT": 309.45}
    return lows.get(ticker.upper(), 0.0)

print("Three tools ready:")
print("- get_stock_price")
print("- get_52_week_high")
print("- get_52_week_low")

Three tools ready:
- get_stock_price
- get_52_week_high
- get_52_week_low


---

## The Multi-Step Query

Now let's ask that complex question and watch the ReAct loop in action.

**Key:** We're using `verbosity_level=LogLevel.INFO` so you can see the reasoning.

In [5]:
agent = CodeAgent(
    tools=[get_stock_price, get_52_week_high, get_52_week_low],
    model=model,
    verbosity_level=LogLevel.INFO  # So we can see the reasoning!
)

# With clear, step by step instructions
result = agent.run("""
Compare Apple and NVIDIA:
- What's each stock's current price?
- What's each stock's 52-week high?
- Which stock is trading closer to its 52-week high (as a percentage)?
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Compare Apple and NVIDIA:                                                                                       │
│ - What's each stock's current price?                                                                            │
│ - What's each stock's 52-week high?                                                                             │
│ - Which stock is trading closer to its 52-week high (as a percentage)?                                          │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  apple_price = get_stock_price(ticker='AAPL')                                                                     
  nvidia_price = get_stock_price(ticker='NVDA')                                                                    
  apple_52_week_high = get_52_week_high(ticker='AAPL')                                                             
  nvidia_52_week_high = get_52_week_high(ticker='NVDA')                                                            
                                                                                                                   
  print(f"Apple Price: {apple_price}, NVIDIA Price: {nvidia_price}")                                               
  print(f"Apple 52-week High: {apple_52_week_high}, NVIDIA 52-week High: {nvidia_52_week_high}")                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Apple Price: 178.5, NVIDIA Price: 875.3
Apple 52-week High: 199.62, NVIDIA 52-week High: 974.0

Out: None

[Step 1: Duration 5.82 seconds| Input tokens: 2,169 | Output tokens: 232]

...

[Step 2: Duration 5.50 seconds| Input tokens: 4,784 | Output tokens: 468]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "Apple": {                                                                                                   
          "Current Price": apple_price,                                                                            
          "52 Week High": apple_52_week_high,                                                                      
          "Percentage of 52 Week High": apple_percentage,                                                          
      },                                                                                                           
      "NVIDIA": {                                                                                                  
          "Current Price": nvidia_price,                                                                           
          "52 Week High": nvidia_52_week_high,                                                                     
          "Percentage of 52 Week High": nvidia_percentage,                                                         
      },                                                                                                           
      "Closer Stock": closer_stock                                                                                 
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'Apple': {'Current Price': 178.5, '52 Week High': 199.62, 'Percentage of 52 Week High': 
89.41989780583107}, 'NVIDIA': {'Current Price': 875.3, '52 Week High': 974.0, 'Percentage of 52 Week High': 
89.8665297741273}, 'Closer Stock': 'NVIDIA'}

[Step 3: Duration 4.37 seconds| Input tokens: 7,854 | Output tokens: 668]

In [ ]:
agent = CodeAgent(
    tools=[get_stock_price, get_52_week_high, get_52_week_low],
    model=model,
    verbosity_level=LogLevel.INFO  # So we can see the reasoning!
)

# Works even without clear, step by step instructions
result = agent.run("""
ompare Apple and NVIDIA's stock performance.
 Which one has grown more relative to its 52-week high??
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ ompare Apple and NVIDIA's stock performance.                                                                    │
│  Which one has grown more relative to its 52-week high??                                                        │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  apple_price = get_stock_price(ticker="AAPL")                                                                     
  nvidia_price = get_stock_price(ticker="NVDA")                                                                    
                                                                                                                   
  apple_52_week_high = get_52_week_high(ticker="AAPL")                                                             
  nvidia_52_week_high = get_52_week_high(ticker="NVDA")                                                            
                                                                                                                   
  print(f"Apple Price: {apple_price}, NVIDIA Price: {nvidia_price}")                                               
  print(f"Apple 52-Week High: {apple_52_week_high}, NVIDIA 52-Week High: {nvidia_52_week_high}")                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Apple Price: 178.5, NVIDIA Price: 875.3
Apple 52-Week High: 199.62, NVIDIA 52-Week High: 974.0

Out: None

[Step 1: Duration 4.84 seconds| Input tokens: 2,151 | Output tokens: 240]

...

[Step 2: Duration 3.69 seconds| Input tokens: 4,757 | Output tokens: 434]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("NVIDIA has grown more relative to its 52-week high compared to Apple.")                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: NVIDIA has grown more relative to its 52-week high compared to Apple.

[Step 3: Duration 2.42 seconds| Input tokens: 7,722 | Output tokens: 549]

---

## Anatomy of ReAct

Let's break down what probably happened (results may vary).

```
Step 1: THINK → "I need Apple's price" → ACT → get_stock_price("AAPL") → OBSERVE
Step 2: THINK → "Now Apple's high" → ACT → get_52_week_high("AAPL") → OBSERVE
Step 3: THINK → "Got Apple, now NVIDIA" → ACT → get_stock_price("NVDA") → OBSERVE
Step 4: THINK → "NVIDIA's high" → ACT → get_52_week_high("NVDA") → OBSERVE
Step 5: THINK → "Now I can compare" → Calculate percentages → FINAL ANSWER
```

**Notice:** Each THINK step decides what to do next based on what it's already learned.

The agent isn't following a script. It's **reasoning through the problem dynamically**.

---

## Why ReAct Matters

Three reasons:

### 1. Complex Questions Become Possible
Without ReAct, you'd have to hardcode: first call this, then call that, then combine them. With ReAct, the agent figures out the steps itself.

### 2. Transparency
You can see exactly why the agent did what it did. In finance, this **auditability** matters. You can't use a black box for investment decisions.

### 3. Adaptability
If the question changes — say, "now include Microsoft" — the agent adapts. It doesn't break. It just adds more steps.

---

## Experimenting with ReAct

Let's try a few more queries to see how the agent adapts.

In [6]:
# Query 1: A question that requires handling missing data
result = agent.run("What's the 52-week range for PESLA?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the 52-week range for PESLA?                                                                             │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  high_price = get_52_week_high(ticker="PESLA")                                                                    
  low_price = get_52_week_low(ticker="PESLA")                                                                      
  print(f"52-week high: {high_price}, 52-week low: {low_price}")                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
52-week high: 0.0, 52-week low: 0.0

Out: None

[Step 1: Duration 2.91 seconds| Input tokens: 2,139 | Output tokens: 137]

...

[Step 2: Duration 3.45 seconds| Input tokens: 4,543 | Output tokens: 279]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("No available data for PESLA.")                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: No available data for PESLA.

[Step 3: Duration 3.96 seconds| Input tokens: 7,235 | Output tokens: 444]

**Notice:** The agent reasoned about the zero result. It didn't just say the high is zero." It recognized that zero means missing data.

In [7]:
# Query 2: Investment reasoning
result = agent.run("""
I'm thinking about buying one of these stocks.
Which one is furthest from its 52-week high,
suggesting more potential upside?
Compare AAPL, NVDA, and MSFT.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm thinking about buying one of these stocks.                                                                  │
│ Which one is furthest from its 52-week high,                                                                    │
│ suggesting more potential upside?                                                                               │
│ Compare AAPL, NVDA, and MSFT.                                                                                   │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  tickers = ["AAPL", "NVDA", "MSFT"]                                                                               
  stock_data = {}                                                                                                  
                                                                                                                   
  for ticker in tickers:                                                                                           
      current_price = get_stock_price(ticker)                                                                      
      week_high = get_52_week_high(ticker)                                                                         
      stock_data[ticker] = {'current_price': current_price, 'week_high': week_high}                                
                                                                                                                   
  print(stock_data)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{'AAPL': {'current_price': 178.5, 'week_high': 199.62}, 'NVDA': {'current_price': 875.3, 'week_high': 974.0}, 
'MSFT': {'current_price': 378.9, 'week_high': 420.82}}

Out: None

[Step 1: Duration 3.56 seconds| Input tokens: 2,167 | Output tokens: 172]

...

[Step 3: Duration 3.74 seconds| Input tokens: 7,737 | Output tokens: 563]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("AAPL has the most potential upside with a percentage difference of 10.58% from its 52-week         
  high.")                                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: AAPL has the most potential upside with a percentage difference of 10.58% from its 52-week high.

[Step 4: Duration 2.26 seconds| Input tokens: 11,097 | Output tokens: 647]

The agent doesn't just return numbers. It answers the **actual question**: which has more potential upside based on distance from the high?

That's the **reasoning** part of ReAct.

---

## ReAct Principles

### Principle 1: Ask Complete Questions
Don't just say "AAPL price." Say "What is Apple's current price and how does it compare to its 52-week high?" The more context, the better the reasoning.

### Principle 2: Let the Agent Think
Don't try to micromanage the steps. The agent often finds better approaches than you'd hardcode.

### Principle 3: Watch the Trace
The verbose output isn't just for debugging. It's how you **understand and trust** your agent. If the reasoning looks wrong, the answer probably is too.

---

## Your Turn: Exercise

Ask the agent a multi-step question that requires:
1. Getting data for at least 2 stocks
2. Some kind of comparison or calculation
3. A recommendation or conclusion

**Example ideas:**
- "Which stock has the widest 52-week range (high minus low)?"
- "If I had $10,000 to invest, how many shares of each could I buy?"
- "Which stock is most volatile based on its 52-week range?"

In [ ]:
# EXERCISE: Write your multi-step query here

result = agent.run("""
    YOUR QUERY HERE
""")

---

## Bonus: Track the Steps Programmatically

You can also access the agent's memory to see all the steps it took.

In [ ]:
# Run a query
result = agent.run("Compare the 52-week ranges of AAPL and MSFT")

# Access the memory to see all steps
print("\n" + "="*60)
print("AGENT MEMORY (all steps):")
print("="*60)
for i, step in enumerate(agent.memory):
    print(f"\nStep {i+1}: {type(step).__name__}")
    if hasattr(step, 'content'):
        print(f"  Content: {str(step.content)[:200]}...")

---

## Recap

**What you learned:**

1. **ReAct** is Reason + Act — the agent alternates between thinking and doing
2. The **Think → Act → Observe** loop continues until the agent has enough information
3. **Transparency** is built in — you can see every step of the reasoning
4. **Complex questions** become possible because the agent figures out the steps dynamically

**Next up:** In Module 3, we'll see what happens when the tools aren't enough — when the agent needs to generate and run custom code. That's the **CodeAct pattern**.